In [1]:
!pip install -r requirements.txt


  Using cached pymongo-4.8.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (22 kB)
  Using cached rbo-0.1.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached dnspython-2.6.1-py3-none-any.whl.metadata (5.8 kB)
Using cached pymongo-4.8.0-cp312-cp312-macosx_11_0_arm64.whl (698 kB)
Using cached rbo-0.1.3-py3-none-any.whl (7.8 kB)
Using cached dnspython-2.6.1-py3-none-any.whl (307 kB)


In [2]:
"""
Load the Libraries

"""


import numpy as np
import functools
from scipy.sparse import csr_matrix
import pickle
import rbo
from collections import Counter

"""
Load the Project File Data

"""
filename = "project_author_files_dict.pkl"
mongo_db = np.load("mongo_top_200.npy")

In [3]:
#-----------------------------------------------------------------------------------------


def row_normalize(M):
    U = M
    for i in range(U.shape[0]):
        rowsum = U[i].sum()
        if rowsum > 0:
            U[i] = U[i] / rowsum
    return U

def custom_sort(a, b):
    if a[0]>b[0]:
        return -1
    elif a[0]==b[0] and a[1]<b[1]:
        return -1
    return 1


def repeated_items(lst,val):
    counts = Counter(lst)
    top_values = counts.most_common(val)
    return top_values

#-----------------------------------------------------------------------------------------
"""
Load the Project File Data

"""


# filename = "/content/drive/MyDrive/Thesis_Codes/project_author_files_dict.pkl"
# Load the data from the pickle file
with open(filename, "rb") as f:
    project_author_file_dict = pickle.load(f)
print("Dictionary loaded from pickle file:")
# print(project_author_file_dict)

def create_df_matrix(project_author_file_dict):

  unique_developers = set()
  all_files = []

  for project, authors_files in project_author_file_dict.items():
    for author, files in authors_files.items():
      unique_developers.add(author)
      all_files.extend(files)  # Extend the list with unique files from current project

  # Remove duplicates from all_files to ensure uniqueness
  unique_files = list(set(all_files))

  developer_index = {developer: idx for idx, developer in enumerate(unique_developers)}
  file_index = {file: idx for idx, file in enumerate(unique_files)}

  rows = []
  cols = []
  for project, authors_files in project_author_file_dict.items():
    for author, files in authors_files.items():
      dev_idx = developer_index[author]
      for file in files:
        file_idx = file_index[file]
        rows.append(dev_idx)
        cols.append(file_idx)

  df_matrix = csr_matrix((np.ones(len(rows)), (rows, cols)), shape=(len(unique_developers), len(unique_files)))
  return df_matrix, developer_index, file_index, all_files

# mongo_db = np.load("/content/drive/MyDrive/Thesis_Codes/mongo_top_200.npy")

def HRank_ASym_CoRanking_Modified(A, B, UAP, UP, MPUPC, thres ,alpha=0.20, beta=0.8, gamma=0.2):

  # Restart vector
  E_restart1 = np.full(len(A), 1/len(A))
  E_restart2 = np.full(len(B), 1/len(B))

  # Initial rank
  Vis_Prob1 = E_restart1.reshape(1, -1)
  Vis_Prob2 = E_restart2.reshape(1, -1)

  # Calculate MCP
  MCP = np.dot(UAP, np.dot(UP, MPUPC))

  # HRank iterations
  prev_iter1 = np.full(len(A), 1)
  prev_iter2 = np.full(len(B), 1)

  cn = 0
  while True:
    Vis_Prob1_copy = Vis_Prob1.copy()
    Vis_Prob2_copy = Vis_Prob2.copy()

    Vis_Prob1 = np.power(alpha * np.dot(Vis_Prob1_copy, MCP) + (1 - alpha) * E_restart1, gamma)
    Vis_Prob1 = Vis_Prob1 / np.sum(Vis_Prob1)  # Normalize
    Vis_Prob2 = np.power(beta * np.dot(Vis_Prob2_copy, MCP.T) + (1 - beta) * E_restart2, gamma)
    Vis_Prob2 = Vis_Prob2 / np.sum(Vis_Prob2)  # Normalize

    # Check for convergence
    diff = max(np.max(np.absolute(np.nan_to_num(prev_iter1 - Vis_Prob1))), np.max(np.absolute(np.nan_to_num(prev_iter2 - Vis_Prob2))))
    if diff < 0.0001:
      break

    prev_iter1 = Vis_Prob1
    prev_iter2 = Vis_Prob2
    cn += 1

  combined_scores = np.log(Vis_Prob1 * Vis_Prob2.T + 1e-10)  # Add a small value to avoid log(0)
  combined_scores = combined_scores - np.min(combined_scores)  # Normalize

  # Sort and extract co-ranked pairs
  rank = sorted(zip(combined_scores.flatten(), zip(A, B)), key=functools.cmp_to_key(custom_sort))

  # Retrieve co-ranked pairs
  coranked_pairs = [(dev, rev) for (_, (dev, rev)) in rank]
  filtered_pairs = [pair for pair in coranked_pairs if pair[0] != pair[1]]

  return coranked_pairs, filtered_pairs




for mm in range(1,77):

    dict_items = list(project_author_file_dict.items())
    # Take the first 20 elements from the list
    first_20_elements = dict_items[mm-1:mm]
    # Convert the sliced list back into a dictionary
    first_20_dict = dict(first_20_elements)

    # Call the function
    df_matrix, developer_index, file_index , all_files= create_df_matrix(first_20_dict)

    developers = list(developer_index.keys())

    my_list = all_files
    top_files = repeated_items(my_list,len(developers))

    reviewers = top_files

    M_p1 = np.random.rand(len(reviewers), len(developers))
    M_p2 = np.random.rand(len(developers), len(reviewers))

    # Call the HRank_ASym function with the extracted information
    coranked_pairs, filtered_pairs = HRank_ASym_CoRanking_Modified(developers, reviewers, M_p1, M_p2 ,M_p2,thres=200)

    # Retrieve names and IDs of developers from the developer_index dictionary
    developers_info = [(developer_index[developer], developer) for developer, reviewer in coranked_pairs]


    corank_sorted_data = sorted(coranked_pairs, key=lambda x: x[1][1],reverse=True)

    top_files_copair = []
    for i in corank_sorted_data:
        top_files_copair.append(str(i[1][0]))

    mongo_db_list= []
    for i in mongo_db:
        mongo_db_list.append(str(i))


    S = top_files_copair[:200]
    T = mongo_db_list

    print(f"Similarity of RBO value project {mm} ---> {rbo.RankingSimilarity(S, T).rbo()}")


Dictionary loaded from pickle file:
Similarity of RBO value project 1 ---> 0.0
Similarity of RBO value project 2 ---> 0.0
Similarity of RBO value project 3 ---> 0.5560402645365423
Similarity of RBO value project 4 ---> 0.18372902243460795
Similarity of RBO value project 5 ---> 0.0
Similarity of RBO value project 6 ---> 0.0
Similarity of RBO value project 7 ---> 0.0
Similarity of RBO value project 8 ---> 0.0
Similarity of RBO value project 9 ---> 0.0
Similarity of RBO value project 10 ---> 0.06865726037363465
Similarity of RBO value project 11 ---> 0.0
Similarity of RBO value project 12 ---> 0.0
Similarity of RBO value project 13 ---> 0.05206344643245155
Similarity of RBO value project 14 ---> 0.04770873124768239
Similarity of RBO value project 15 ---> 0.015011978986017256
Similarity of RBO value project 16 ---> 0.012036148793517968
Similarity of RBO value project 17 ---> 0.0
Similarity of RBO value project 18 ---> 0.0
Similarity of RBO value project 19 ---> 0.0
Similarity of RBO value 